In [18]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import numpy as np
import pandas as pd
import os
import openpyxl
import xlwings as xw
os.chdir("/home/workshop/Huashubei-2025")
mode1 = "Lucas et al., 2014"
mode2 = "CIE S 026"

#Lucas version
def get_mel_DER_Lucas(spd: pd.DataFrame):
    
    #power numpy gen
    power = spd["power"].to_numpy()
    
    #path
    input_excel = "Data/CIE S 026 alpha-opic Toolbox Lucas.xlsx"
    output_file = "Data/CIE S 026 alpha-opic Toolbox_modified.xlsx"
    
    # 加载Excel文件
    wb = openpyxl.load_workbook(input_excel, data_only=True)
    inputs_sheet = wb['Inputs']
    
    # 将numpy数据填充到C24到C424单元格
    for i, value in enumerate(power):
        inputs_sheet[f'C{24 + i}'] = float(value)
    
    # 保存修改后的Excel文件
    wb.save(output_file)
    
    # 读取Advanced Outputs工作表中的J40值
    advanced_outputs_sheet = wb['Advanced Outputs']
    j40_value = advanced_outputs_sheet['J40'].value
    
    return j40_value

#CIE version
def get_mel_DER_CIE(spd: pd.DataFrame):
    
    #power numpy gen
    power = spd["power"].to_numpy()
    
    #path
    input_excel = "Data/CIE S 026 alpha-opic Toolbox CIE.xlsx"
    output_file = "Data/CIE S 026 alpha-opic Toolbox_modified.xlsx"
    
    # 加载Excel文件
    wb = openpyxl.load_workbook(input_excel, data_only=True)
    inputs_sheet = wb['Inputs']
    
    # 将numpy数据填充到C24到C424单元格
    for i, value in enumerate(power):
        inputs_sheet[f'C{24 + i}'] = float(value)
    
    # 保存修改后的Excel文件
    wb.save(output_file)
    
    # 读取Advanced Outputs工作表中的J40值
    advanced_outputs_sheet = wb['Advanced Outputs']
    j40_value = advanced_outputs_sheet['J40'].value
    
    return j40_value


def get_mel_DER_CIE_with_xlwings(spd: pd.DataFrame, input_excel="Data/CIE S 026 alpha-opic Toolbox CIE.xlsx"):
    # 启动Excel应用（可见，调试时方便，发布时可设为False）
    app = xw.App(visible=False)
    try:
        wb = app.books.open(input_excel)
        sheet_inputs = wb.sheets['Inputs']
        sheet_outputs = wb.sheets['Advanced Outputs']

        # power数组
        power = spd['power'].to_list()

        # 从C24开始填充功率数据
        start_row = 24
        col = 'C'
        for i, val in enumerate(power):
            cell = f"{col}{start_row + i}"
            sheet_inputs.range(cell).value = float(val)

        # 强制Excel计算
        wb.app.calculate()

        # 读取J40单元格的计算结果
        j40_value = sheet_outputs.range('J40').value

        wb.close()
        return j40_value
    finally:
        app.quit()


ModuleNotFoundError: No module named 'xlwings'

In [27]:
wavelengths = np.arange(380, 781, 1)
power = np.random.rand(len(wavelengths))
dummy_df = pd.DataFrame({
    'wavelength': wavelengths,
    'power': power
})

result = get_mel_DER_CIE(dummy_df)
print(f"从Advanced Outputs的J40读取的值是: {result}")

从Advanced Outputs的J40读取的值是: #VALUE!
